# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Handig of om te testen of om het resultaat van bepaalde stappen te bekijken.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

### Instructies

1. Kopieer de `DR`, `DR Extra`, `FR`, `FR Extra` en `GR` folders uit `/Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `../testing/vegetatiekarteringen`.
   - Hiervan worden enkel `/GR/SGL Hunzedal en Leekstermeer2021` en `/FR/Rottige_Meenthe_Brandemeer` gebruikt, maar de rest is nodig voor `alle_karteringen_door_veg2hab.ipynb`.

2. Er worden zowel een shapefilekartering als een digitale standaard kartering ingeladen; welke vervolgens omgezet wordt kan aangepast worden onder het kopje *Selecteren welke kartering omgezet moet worden*

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.io.cli import CLIInterface
import pandas as pd
from veg2hab.fgr import FGR

CLIInterface.get_instance()

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

Alle SBB codes zijn valide
Alle VvN codes zijn valide


,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,mitsjson,mozaiekjson,Criteria,Mozaiekregel
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
3,5,H1110_B,Kwaliteit.GOED,50a,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
4,6,H1110_B,Kwaliteit.GOED,50b,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)


### FGR inladen

Bij het inladen worden de FGR typen omgezet naar de FGRType Enum, en hierdoor vind ook validatie plaats.

In [4]:
fgr = FGR(Path("../data/bronbestanden/FGR.json"))

## Inladen van een kartering vanaf een shapefile

In [5]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
shapefile_kartering = Kartering.from_shapefile(shp_path, vegtype_col_format="single", sbb_of_vvn="sbb", ElmID_col="elmid", SBB_col="SBBTYPE", split_char="+", lok_vegtypen_col="Vegtype").gdf

## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [6]:
shape_path = Path("../testing/vegetatiekarteringen/FR/Rottige_Meenthe_Brandemeer/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../testing/vegetatiekarteringen/FR/Rottige_Meenthe_Brandemeer/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


mdbtools v1.0.0


,ElmID,geometry,Opmerking,Datum,Opp,_LokVrtNar,intern_id,Locatie_x,VegTypeInfo,Locatie_y,_LokVegTyp
0,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",None,None,3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9...",1809,"09a2-6 (90%), 09a2-1 (10%)"
1,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",None,None,10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]",815,50a-1 (100%)
2,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",None,None,2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9...",650,"09a2-1 (90%), 09a2-6 (10%)"


## Selecteren welke kartering omgezet moet worden

In [7]:
kartering = access_kartering
# kartering = shapefile_kartering

#### Opzoeken en toevoegen VvN aan kartering

In [8]:
kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3')]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [9]:
kartering.apply_deftabel(dt)

### De mitsen en mozaiekregels checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [10]:
kartering.bepaal_habitatkeuzes(fgr)
kartering.gdf.head()

Iteratie 0: van 3499 naar 101 keuzes nog te bepalen
Iteratie 1: van 101 naar 40 keuzes nog te bepalen
Iteratie 2: van 40 naar 35 keuzes nog te bepalen
Iteratie 3: van 35 naar 33 keuzes nog te bepalen
Iteratie 4: van 33 naar 33 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1079: UserWarning: Er zijn nog 33 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


,ElmID,geometry,Opmerking,Datum,Opp,_LokVrtNar,intern_id,Locatie_x,VegTypeInfo,Locatie_y,_LokVegTyp,HabitatVoorstel,HabitatKeuze,finished_on_iteration
0,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",None,None,3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",1809,"09a2-6 (90%), 09a2-1 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
1,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",None,None,10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]",815,50a-1 (100%),[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.NIET_GEAUTOM...,1
2,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",None,None,2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",650,"09a2-1 (90%), 09a2-6 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
3,15075,"POLYGON ((190801.313 539561.740, 190819.617 53...",None,None,18878.050297,Lokale typologie is primair vertaald naar SBB,2515,2515,"[(80%, SBB: ['9b-a'], VvN: []), (10%, SBB: ['9...",2515,"09b-4 (80%), 09-13 (10%), 09a2-2 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.GEEN_KLOPPEN...,1
4,20502,"POLYGON ((190497.231 537724.945, 190505.062 53...",None,None,4950.611630,Lokale typologie is primair vertaald naar SBB,1316,1316,"[(58%, SBB: ['8b3c'], VvN: ['8bb4d']), (29%, S...",1316,"08b3-3 (58%), 08c-6 (29%), 09-11 (10%), 05d5-1...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.VEGTYPEN_NIE...,1


### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [11]:
final_format = kartering.as_final_format()
final_format.head()

,Area,Opm,Datum,ElmID,geometry,_Samnvttng,_LokVegTyp,_LokVrtNar,Habtype1,Perc1,...,SBB5,VvN5,_VgTypInf5,_Status5,_Uitleg5,_SBBdftbl5,_VvNdftbl5,_Mits_opm5,_Mozk_opm5,_MozkPerc5
0,3761.736281,None,None,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",100.0% H7140_B,"09a2-6 (90%), 09a2-1 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10442.658431,None,None,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",100.0% HXXXX,50a-1 (100%),Lokale typologie is primair vertaald naar SBB,HXXXX,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2174.098680,None,None,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",100.0% H7140_B,"09a2-1 (90%), 09a2-6 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18878.050297,None,None,15075,"POLYGON ((190801.313 539561.740, 190819.617 53...","80.0% H0000, 10.0% H7140_B, 10.0% HXXXX","09b-4 (80%), 09-13 (10%), 09a2-2 (10%)",Lokale typologie is primair vertaald naar SBB,HXXXX,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4950.611630,None,None,20502,"POLYGON ((190497.231 537724.945, 190505.062 53...","61.0% H0000, 39.0% HXXXX","08b3-3 (58%), 08c-6 (29%), 09-11 (10%), 05d5-1...",Lokale typologie is primair vertaald naar SBB,HXXXX,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))